In [ ]:
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (8, 8)
plt.rcParams["font.size"] = 15
from matplotlib import cm
from matplotlib import colors
import datetime as dt
pd.options.display.max_columns = None

In [ ]:
datos=[]
ataque=pd.read_csv('data/attacking.csv').drop(['serial','match_played'],axis=1)
intentos=pd.read_csv('data/attempts.csv').drop(['serial','match_played'],axis=1)
defenza=pd.read_csv('data/defending.csv').drop(['serial','match_played'],axis=1)
disciplina=pd.read_csv('data/disciplinary.csv').drop(['serial','match_played','minutes_played'],axis=1)
pases=pd.read_csv('data/distributon.csv').drop(['serial','match_played'],axis=1)
porteros=pd.read_csv('data/goalkeeping.csv').drop(['serial','match_played'],axis=1)
goles=pd.read_csv('data/goals.csv').drop(['serial','match_played'],axis=1)
general=pd.read_csv('data/key_stats.csv').drop(['goals','assists'],axis=1)
datos=[intentos,defenza,disciplina,pases,porteros,goles,general]
df=ataque
for i in datos:
    df=pd.merge(df,i,on=['player_name','club','position'],how='outer')
df.fillna(0,inplace=True)
df.drop(df[df['player_name']=='Reinildo'].index,inplace=True)
df.drop(df[df['player_name']=='Luis Díaz'].index,inplace=True)
df.drop(df[df['player_name']=='Brobbey'].index,inplace=True)
df.to_csv('resultados.csv')

In [ ]:
df.columns

In [ ]:
df.head(15)

# Datos generales
Se presentan las estadisticas totales de la temporada

In [ ]:
totales=df['goals'].sum()
izquierda=round(df['left_foot'].sum()*100/totales,2)
derecha=round(df['right_foot'].sum()*100/totales,2)
cabeza=round(df['headers'].sum()*100/totales,2)
otros=round(df['others'].sum()*100/totales,2)
dentro_del_area=round(df['inside_area'].sum()*100/totales,2)
fuera_del_area=round(df['outside_areas'].sum()*100/totales,2)

In [ ]:

print('Cantidad de equipos: {}'.format(len(pd.unique(df['club']))))
print('Cantidad de jugadores: {}'.format(len(pd.unique(df['player_name']))))
print("---------------------------------------------------")
print('Cantidad de tarjetas amarillas: {}'.format(int(df['yellow'].sum())))
print('Cantidad de tarjetas rojas: {}'.format(int(df['red'].sum())))
print("---------------------------------------------------")
print('Asistencias totales: {}'.format(int(df['assists'].sum())))
print('Goles totales: {}'.format(totales))
print("---------------------------------------------------")
print('Goles con la izquierda: {} --> %{}'.format(int(df['left_foot'].sum()),izquierda))
print('Goles con la derecha: {} --> %{}'.format(int(df['right_foot'].sum()),derecha))
print('Goles con la cabeza: {} --> %{}'.format(int(df['headers'].sum()),cabeza))
print('Otros: {} --> %{}'.format(int(df['others'].sum()),otros))
print("---------------------------------------------------")
print('Goles dentro del area: {} --> %{}'.format(int(df['inside_area'].sum()),dentro_del_area))
print('Goles fuera del area: {} --> %{}'.format(int(df['outside_areas'].sum()),fuera_del_area))
print("---------------------------------------------------")
print('Goles de penal: {}'.format(int(df['penalties'].sum())))
print("---------------------------------------------------")

In [ ]:
datos=[derecha,izquierda,cabeza,otros]
normdata=colors.Normalize(min(datos),max(datos))
colormap = cm.get_cmap("jet")
colores =colormap(normdata(datos))
separacion=[0.02,0.02,0.02,0.02]
fig, ax = plt.subplots()
ax.pie(datos,colors=colores, explode=separacion,labels=['Derecha','Izquierda','Cabeza','Otros'], autopct="%0.1f %%",labeldistance=1.1,textprops={'color':"azure"})
ax.axis('equal')
plt.title('Goles temporada 2021',y=1.05,fontdict={'fontsize':20,'color':'azure', 'fontweight':'bold'})
plt.show()

In [ ]:
datos=[dentro_del_area,fuera_del_area]
normdata=colors.Normalize(min(datos),max(datos))
colormap = cm.get_cmap("turbo")
colores =colormap(normdata(datos))
separacion=[0.02,0.02]
fig, ax = plt.subplots()
ax.pie(datos,colors=colores, explode=separacion,labels=['Dentro del area','Fuera del area'], autopct="%0.1f %%",labeldistance=1.1,textprops={'color':"azure"})
ax.axis('equal')
plt.title('Goles temporada 2021',y=1.05,fontdict={'fontsize':20,'color':'azure', 'fontweight':'bold'})
plt.show()

# Goleadores

In [ ]:
df[['player_name','club','match_played','goals']].sort_values('goals',ascending=False).reset_index(drop=True).style.background_gradient("Greens")